# Accelerating End-to-End Data Science Workflows #

## 08 - K-Means แบบ Multi-GPU ด้วย Dask ##

**สารบัญ**
<br>
สมุดบันทึกนี้ใช้ K-means ที่เร่งความเร็วด้วย GPU เพื่อระบุกลุ่มประชากรในลักษณะที่ปรับขนาดได้แบบหลายโหนด (multi-node) และหลาย GPU (multi-GPU) ด้วย Dask สมุดบันทึกนี้ครอบคลุมส่วนต่างๆ ดังนี้:
1. [สภาพแวดล้อม](#Environment)
2. [การโหลดและคงข้อมูล](#Load-and-Persist-Data)
3. [การฝึกโมเดล](#Training-the-Model)
    * [แบบฝึกหัดที่ 1 - นับจำนวนสมาชิกของคลัสเตอร์ที่อยู่ทางใต้สุด](#Exercise-#1---Count-Members-of-the-Southernmost-Cluster)

## สภาพเเวดล้อม ##

ขั้นแรก เราจะนำเข้า **โมดูลที่จำเป็น** เพื่อสร้างคลัสเตอร์ **Dask cuDF** เหมือนที่เราเคยทำมาก่อน เราต้องนำเข้า **CUDA context creators** หลังจากตั้งค่าคลัสเตอร์แล้ว เพื่อที่มันจะไม่ล็อกอยู่กับอุปกรณ์เพียงตัวเดียว

In [ ]:
import subprocess
import logging

from dask.distributed import Client, wait, progress
from dask_cuda import LocalCUDACluster

In [ ]:
import cudf
import dask_cudf

import cuml
from cuml.dask.cluster import KMeans

In [ ]:
# create cluster
cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

cluster = LocalCUDACluster(ip=IPADDR, silence_logs=logging.ERROR)
client = Client(cluster)

## โหลดและคงข้อมูล (Load and Persist Data) ##
เราจะเริ่มต้นด้วยการโหลดข้อมูล ชุดข้อมูลมีคอลัมน์พิกัดกริดสองคอลัมน์ ได้แก่ **`easting`** และ **`northing`** ซึ่งได้มาจากชุดข้อมูลประชากรหลักที่เราได้จัดเตรียมไว้

In [ ]:
ddf = dask_cudf.read_csv('./data/uk_pop5x_coords.csv', dtype=['float32', 'float32'])

## การฝึกโมเดล (Training the Model) ##
การฝึกโมเดล K-means มีความคล้ายคลึงอย่างมากกับเวอร์ชัน scikit-learn และ cuML เวอร์ชัน GPU เดี่ยว โดยการตั้งค่าไคลเอนต์และนำเข้าจากโมดูล `cuml.dask.cluster` อัลกอริทึมจะใช้คลัสเตอร์ Dask ในเครื่องที่เราได้ตั้งค่าไว้โดยอัตโนมัติ

โปรดทราบว่าการเรียกใช้ `.fit` จะเป็นการเรียกให้ Dask ทำการคำนวณ

เมื่อเราได้โมเดลที่ผ่านการฝึกแล้ว เราจะดึงจุดศูนย์กลางของคลัสเตอร์ (cluster centers) ออกมา และเปลี่ยนชื่อคอลัมน์จากชื่อทั่วไปอย่าง `0` และ `1` เพื่อให้สอดคล้องกับข้อมูลที่ใช้ในการฝึก


In [ ]:
%%time
dkm = KMeans(n_clusters=20)
dkm.fit(ddf)

In [ ]:
cluster_centers = dkm.cluster_centers_
cluster_centers.columns = ddf.columns
cluster_centers.dtypes

### แบบฝึกหัดที่ 1 - นับสมาชิกของกลุ่มที่อยู่ใต้สุด ###
ใช้ `cluster_centers` เพื่อระบุว่ากลุ่มใดอยู่ทางใต้สุด (มีค่า `northing` ต่ำที่สุด) ด้วยเมธอด `nsmallest` จากนั้นใช้ `dkm.predict` เพื่อรับป้ายกำกับสำหรับข้อมูล และสุดท้ายกรองป้ายกำกับเพื่อดูว่าแบบจำลองประมาณจำนวนบุคคลในกลุ่มนั้นมีจำนวนเท่าใด

**คำแนะนำ**: <br>
* แก้ไขเฉพาะ `<FIXME>` และรันเซลล์ด้านล่างเพื่อประมาณจำนวนบุคคลในกลุ่มที่อยู่ใต้สุด

In [ ]:
south_idx = cluster_centers.nsmallest(1, 'northing').index[0]
labels_predicted = dkm.<<<<FIXME>>>>(<<<<FIXME>>>>)
labels_predicted[labels_predicted==south_idx].compute().shape[0]

Click ... for solution. 

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!**